In [3]:
import getpass
import json
import uuid

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import (check_premises,
                                   execute_query,
                                   execute_query_pipe,
                                   apply_rule,
                                   get_rule_parameters)

In [4]:
LOCAL_TOKEN = True 
TOKEN_PATH = '../../../token.txt'
CONFIG_PATH = "../../../config.yaml"

def allocate_forge_session(org, project):
    ENDPOINT = "https://bbp.epfl.ch/nexus/v1"  

    if LOCAL_TOKEN: 
        with open(TOKEN_PATH) as f:
            TOKEN = f.read()
    else: 
        TOKEN = getpass.getpass()
        
    searchendpoints = {
        "sparql": {
           "endpoint": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
       }
    }
    
    return KnowledgeGraphForge(
        CONFIG_PATH,
        endpoint=ENDPOINT,
        token=TOKEN, 
        bucket=f"{org}/{project}", 
        searchendpoints=searchendpoints,
        debug=True
    )

# [Create a Rule able to Generalize data across the brain region hierarchy](https://bbpteam.epfl.ch/project/issues/browse/DKE-1105)

In [5]:
input_filters = {
    "BrainRegionQueryParameter": "<http://api.brain-map.org/api/v2/data/Structure/315>",
    "TypeQueryParameter": "<https://neuroshapes.org/NeuronMorphology>"
}

## V1: QueryPipe: 
- 1. Get all Neuron morphologies 
- 2. Only keep those in the right brain region

#### Problem: timing out, too many neuron morphologies retrieved and passed to the next step?

In [6]:
rule_id = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"


fq =   """
    SELECT ?id ?br
    WHERE { 
        ?id rdf:type $TypeQueryParameter .
        ?id nsg:brainLocation/nsg:brainRegion ?br .
        
        ?id <https://bluebrain.github.io/nexus/vocabulary/deprecated> ?_deprecated .
        Filter (?_deprecated = 'false'^^xsd:boolean)
        } 
"""

fqo = {
        "type": "SparqlQuery",
        "hasBody": fq,
        "hasParameter": [
            {
                "type": "uri",
                "description": "type of the entity being retrieved",
                "name": "TypeQueryParameter"
            },
        ],
        "queryConfiguration": {
            "sparlqlView": {
                "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
            },
            "org": "bbp",
            "project": "atlas"
        },
        "resultParameterMapping": [
            {
                "parameterName": "entityBr",
                "path": "br"
            },
            {
                "parameterName": "entity",
                "path": "id"
            }
        ]
    }

sq = """
    SELECT ?id ?br
    WHERE {
     ?br rdfs:subClassOf BrainRegion ;   
     <http://schema.org/isPartOf>* $BrainRegionQueryParameter.
        VALUES (?id) {
            $entity
        }.
        VALUES (?br){
            $entityBr
        }
    }
"""

sqo = {
    "type": "SparqlQuery",
    "hasBody": sq,
    "hasParameter": [
    {
        "type": "sparql_value_list",
        "description": "List of URIs of brains regions",
        "name": "entityBr"
    }, 
    {
        "type": "sparql_value_list",
        "description": "List of URIs of entities of type",
        "name": "entity"
    }, 
    {
        "type": "uri",
        "description": "URI of the queried brain region",
        "name": "BrainRegionQueryParameter"
    }
    ],
    "queryConfiguration": {
        "sparlqlView": {
            "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
        },
        "org": "neurosciencegraph",
        "project": "datamodels"
    },
    "resultParameterMapping": {
        "parameterName": "test",
        "path": "entity"
    }
}


my_rule = {
    "id": rule_id,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": "Generalise up (ancestors) and/or down (descendants) a (combination of)"+
    "hierarchy in a BBP ontology (e.g. cell type, brain region)",
    "name": "TODO", #TODO
    "searchQuery": {
        "type": "QueryPipe",
        "head": fqo, 
        "rest": sqo
    }
}

In [14]:
# res = apply_rule(allocate_forge_session, my_rule, input_filters)
# res

## QueryPipe: 
- 1. Get all valid brain regions
- 2. Query Neuron Morphologies and keep only those with the valid brain regions

In [33]:
rule_id2 = f"https://bbp.epfl.ch/neurosciencegraph/data/{uuid.uuid4()}"


fq2 = """
    SELECT ?id
    WHERE {
     ?id rdfs:subClassOf BrainRegion ;   
     <http://schema.org/isPartOf>* $BrainRegionQueryParameter.
    }
"""

fqo2 = {
        "type": "SparqlQuery",
        "hasBody": fq2,
        "hasParameter": [
            {
                "type": "uri",
                "description": "type of the brain region it should belong to",
                "name": "BrainRegionQueryParameter"
            },
        ],
        "queryConfiguration": {
            "sparlqlView": {
                "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
            },
            
            "org": "neurosciencegraph",
            "project": "datamodels"
        },
        "resultParameterMapping": [
            {
                "parameterName": "all_brs",
                "path": "id"
            },
        ]
    }


sq2 =   """
    SELECT ?id ?br
    WHERE { 
        ?id rdf:type $TypeQueryParameter .
        ?id nsg:brainLocation/nsg:brainRegion ?br .
        ?id <https://bluebrain.github.io/nexus/vocabulary/deprecated> ?_deprecated .
        Filter (?_deprecated = 'false'^^xsd:boolean &&  ?br IN $all_brs)
    } 
"""

sqo2 = {
    "type": "SparqlQuery",
    "hasBody": sq2,
    "hasParameter": [
    {
        "type": "sparql_list",
        "description": "List of URIs of brains regions",
        "name": "all_brs"
    }, 
    {
        "type": "uri",
        "description": "type of the queried entity",
        "name": "TypeQueryParameter"
    }
    ],
    "queryConfiguration": {
        "sparlqlView": {
            "id": "https://bbp.epfl.ch/neurosciencegraph/data/views/aggreg-sp/dataset"
        },
        "org": "bbp",
        "project": "atlas"
    },
    "resultParameterMapping": {
        "parameterName": "test",
        "path": "entity"
    }
}


my_rule2 = {
    "id": rule_id2,
    "type": "DataGeneralizationRule",
    "_schemaProject": "https://bbp.epfl.ch/nexus/v1/projects/bbp/inference-rules",
    "description": """
        Given an entity type (e.g NeuronMorphology, Trace, cell type, ...) 
        linked with a given brain region (e.g. Isocortex), generalise to entities associated 
        with the descendants of the region (e.g Gustatory areas)
    """,
    "name": """
        Generalise up (ancestors) and/or down (descendants) a (combination of) 
        "hierarchy in a BBP ontology (e.g. cell type, brain region)
    """,
     "searchQuery": {
        "type": "QueryPipe",
        "head": fqo2, 
        "rest": sqo2
    }
   
}

In [9]:
res2 = apply_rule(allocate_forge_session, my_rule2, input_filters)
len(res2)

619

# Expected

## 1. Get all neuron morphologies

In [13]:
bbp_atlas_forge = allocate_forge_session("bbp", "atlas")

In [6]:
# !cat ../../../inference_test/lib/python3.7/site-packages/..

### a. SPARQL - KG_Inference - All but deprecated

In [9]:
first_query_res = execute_query(allocate_forge_session, fqo, input_filters, last_query=True)

Submitted query:
   PREFIX bmc: <https://bbp.epfl.ch/ontologies/core/bmc/>
   PREFIX bmo: <https://bbp.epfl.ch/ontologies/core/bmo/>
   PREFIX commonshapes: <https://neuroshapes.org/commons/>
   PREFIX datashapes: <https://neuroshapes.org/dash/>
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>
   PREFIX oa: <http://www.w3.org/ns/oa#>
   PREFIX obo: <http://purl.obolibrary.org/obo/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFI

In [10]:
len(first_query_res)

3295

### b. SPARQL - Forge

In [16]:
DEBUG = False

The following 3 numbers are true only if commenting out line 372 of [kgforge/core/archetypes/store.py](
https://github.com/BlueBrain/nexus-forge/blob/9e62fb4a8ec8db97366fa6b6de75dadccaf02967/kgforge/core/archetypes/store.py)

In [59]:
! sed -n 363,379p ../../../inference_test/lib/python3.7/site-packages/kgforge/core/archetypes/store.py

    def sparql(
        self, query: str, debug: bool, limit: int = None, offset: int = None, **params
    ) -> List[Resource]:
        rewrite = params.get("rewrite", True)
        qr = (
            rewrite_sparql(query, self.model_context, self.service.metadata_context)
            if self.model_context is not None and rewrite
            else query
        )

#         qr = _replace_in_sparql(qr, "LIMIT", limit, 100, r" LIMIT \d+")

        qr = _replace_in_sparql(qr, "OFFSET", offset, 0, r" OFFSET \d+")
        if debug:
            self._debug_query(qr)
        return self._sparql(qr, limit, offset, **params)



In [10]:
LIMIT = None

or a super high limit (10k?)

In [14]:
LIMIT = 10000

#### A. 4380 when getting all included deprecated

In [17]:
qtr = """
    SELECT ?id ?br 
    WHERE { 
        ?id rdf:type <https://neuroshapes.org/NeuronMorphology> .
        ?id nsg:brainLocation/nsg:brainRegion ?br .
    } 
"""
first_query_res = bbp_atlas_forge.as_json(
    bbp_atlas_forge.sparql(qtr, limit=LIMIT, debug=DEBUG))

len(first_query_res)

4380

#### B. 3295 when getting all but deprecated

In [18]:
qtr2 = """
    SELECT ?id ?br 
    WHERE { 
        ?id rdf:type <https://neuroshapes.org/NeuronMorphology> .
        ?id nsg:brainLocation/nsg:brainRegion ?br .
        
        ?id <https://bluebrain.github.io/nexus/vocabulary/deprecated> ?_deprecated .
        Filter (?_deprecated = 'false'^^xsd:boolean)
    } 
"""
first_query_res2 = bbp_atlas_forge.as_json(
    bbp_atlas_forge.sparql(qtr2, limit=LIMIT, debug=DEBUG))
len(first_query_res2)

3295

#### C. 3400 additional ones when getting rev, updatedAt, updatedBy (replicating forge search below)

In [19]:
qtr3 = """
   SELECT ?id ?_deprecated ?_rev 
   WHERE { Graph ?g {?id rdf:type <https://neuroshapes.org/NeuronMorphology>;
    <https://bluebrain.github.io/nexus/vocabulary/deprecated> ?_deprecated;
    <https://bluebrain.github.io/nexus/vocabulary/rev> ?_rev;
    <https://bluebrain.github.io/nexus/vocabulary/updatedAt> ?_updatedAt;
    <https://bluebrain.github.io/nexus/vocabulary/updatedBy> ?_updatedBy . 
    Filter (?_deprecated = 'false'^^xsd:boolean)
   }}
"""
first_query_res3 = bbp_atlas_forge.as_json(
    bbp_atlas_forge.sparql(qtr3, limit=LIMIT, debug=DEBUG))
len(first_query_res3)

3400

### c. Forge Search

In [20]:
first_res = bbp_atlas_forge.search({"type": input_filters["TypeQueryParameter"]}, 
                                   cross_bucket=True, limit=LIMIT, debug=DEBUG)

In [13]:
first_res_format = [{
    "br": el["brainLocation"]["brainRegion"]["id"], 
    "id": el["id"]
} for el in bbp_atlas_forge.as_json(first_res)]

In [14]:
len(first_res_format)

3400

## 2. Get all brain regions that are subclasses* of Isocortex

### SPARQL - Forge

In [23]:
bbp_datamodels_forge = allocate_forge_session("neurosciencegraph", "datamodels")

In [24]:
second_res = bbp_datamodels_forge.sparql("""
    SELECT ?br
    WHERE {
     ?br rdfs:subClassOf BrainRegion .
     ?br <http://schema.org/isPartOf>* <http://api.brain-map.org/api/v2/data/Structure/315> .
    }
""", limit=LIMIT)

In [25]:
second = [el["br"] for el in bbp_datamodels_forge.as_json(second_res)]

In [26]:
len(second)

374

## 3. Keep only the neuron morphologies that have their brain regions as a subclass* of isocortex

### Using the b.B version

In [16]:
first = first_query_res2

In [22]:
merge_two = [el for el in first if el["br"] in second]

In [24]:
len(merge_two)

619

## Same result & approach as V2 rule

# Persisting:

In [34]:
rule_forge = allocate_forge_session("bbp", "inference-rules")

In [35]:
rule_forge.register(rule_forge.from_json(my_rule2))

<action> _register_one
<succeeded> True


In [40]:
my_rule2_res = rule_forge.retrieve(rule_id2)

In [41]:
rule_forge.tag(my_rule2_res, "v1")

<action> _tag_one
<succeeded> True
